# Importing Dependencies

In [1]:
import cv2
import os
from PIL import Image
import numpy as np 

# Generating the Dataset

In [7]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face

    cap = cv2.VideoCapture(0)  # Change to 0 if 1 does not work
    id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            continue
        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1) == 13 or int(img_id) == 200:  # 13 is the ASCII character of Enter
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

generate_dataset()


# Training the classifier and saving it

In [8]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

# Detecting the face and assigning labels :

# 1. In real time

In [2]:
# def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
#     gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
#     for (x,y,w,h) in features:
#         cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
         
#         id, pred = clf.predict(gray_img[y:y+h,x:x+w])
#         confidence = int(100*(1-pred/300))
         
#         if confidence>76:
#             if id==1:
#                 cv2.putText(img, "Michelle", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
# #             if id==2:
# #                 cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
#         else:
#             cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
     
#     return img
 
# # loading classifier
# faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
 
# clf = cv2.face.LBPHFaceRecognizer_create()
# clf.read("classifier.xml")
 
# video_capture = cv2.VideoCapture(0)
 
# while True:
#     ret, img = video_capture.read()
#     img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
#     cv2.imshow("face detection", img)
     
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# video_capture.release()
# cv2.destroyAllWindows()

In [9]:
# import cv2
# import os
# import datetime
# import openpyxl

# def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf, session_id, label_confidence):
#     gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
#     for (x, y, w, h) in features:
#         cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
#         id, pred = clf.predict(gray_img[y:y + h, x:x + w])
#         confidence = int(100 * (1 - pred / 300))
        
#         label = "UNKNOWN"
#         if confidence > 75:
#             if id == 1:
#                 label = "Michelle"
# #             if id == 2:
# #                 label = "Manish"
#         cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
#         label_confidence[label] = confidence
    
#     # Add session ID to the frame
#     cv2.putText(img, f"Session ID: {session_id}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    
#     return img

# def get_next_session_id(sheet):
#     max_id = 0
#     for row in sheet.iter_rows(min_row=2, max_col=1, values_only=True):
#         if row[0] is not None:
#             try:
#                 session_id = int(row[0])
#                 max_id = max(max_id, session_id)
#             except ValueError:
#                 continue
#     return max_id + 1

# def update_log(numeric_session_id, session_id, start_time, end_time, label_confidence):
#     file_name = "observation_logs.xlsx"
    
#     # Load or create the workbook
#     if os.path.exists(file_name):
#         workbook = openpyxl.load_workbook(file_name)
#     else:
#         workbook = openpyxl.Workbook()
#         workbook.active.append(["ID", "Session ID", "Date", "Start Time", "End Time", "Duration (seconds)", "Label", "Confidence Level"])

#     sheet = workbook.active
    
#     # Calculate duration
#     duration = (end_time - start_time).total_seconds()
#     date = start_time.strftime("%Y-%m-%d")
#     start_str = start_time.strftime("%H:%M:%S")
#     end_str = end_time.strftime("%H:%M:%S")
    
#     # Append session data for each label
#     for label, confidence in label_confidence.items():
#         sheet.append([numeric_session_id, session_id, date, start_str, end_str, duration, label, f"{confidence}%"])
    
#     # Save the workbook
#     workbook.save(file_name)

# # Create 'recordings' directory if it doesn't exist
# os.makedirs("recordings", exist_ok=True)

# # Create session ID based on the current timestamp
# start_time = datetime.datetime.now()
# session_id = start_time.strftime("%Y-%m-%d-%H-%M-%S")

# # Create unique filename for recording
# output_path = os.path.join("recordings", f"session_{session_id}.mp4")

# # Define codec and create VideoWriter object
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# fps = 20.0
# frame_size = (640, 480)  # Match this to your webcam's resolution
# out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

# # Loading classifier
# faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# clf = cv2.face.LBPHFaceRecognizer_create()
# clf.read("classifier.xml")

# video_capture = cv2.VideoCapture(0)

# # Dictionary to store label and confidence levels
# label_confidence = {}

# # Load or create the workbook and get the next session ID
# file_name = "observation_logs.xlsx"
# if os.path.exists(file_name):
#     workbook = openpyxl.load_workbook(file_name)
#     sheet = workbook.active
#     numeric_session_id = get_next_session_id(sheet)
# else:
#     numeric_session_id = 1

# while True:
#     ret, raw_frame = video_capture.read()
#     if not ret:
#         break

#     # Resize frame to match the VideoWriter frame size
#     raw_frame = cv2.resize(raw_frame, frame_size)

#     # Create a copy of the frame for processing
#     display_frame = raw_frame.copy()

#     # Process frame (detection and drawing)
#     display_frame = draw_boundary(display_frame, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf, session_id, label_confidence)

#     # Write raw frame to video file (without detection)
#     out.write(raw_frame)

#     # Display the resulting frame with detection
#     cv2.imshow("face detection", display_frame)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Capture end time
# end_time = datetime.datetime.now()

# # Update the observation log
# update_log(numeric_session_id, session_id, start_time, end_time, label_confidence)

# # Release everything
# video_capture.release()
# out.release()
# cv2.destroyAllWindows()


In [8]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf, session_id, label_confidence, frame_collection):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
        
        label = "UNKNOWN"
        flag = "red"
        if confidence > 76:
            if id == 1:
                label = "Michelle"
                flag = "green"
            # Add other known IDs here if necessary
        
        color = (0, 255, 0) if flag == "green" else (0, 0, 255)
        cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        label_confidence[label] = (confidence, flag)
        
        # Collect frames based on confidence
        if label not in frame_collection:
            frame_collection[label] = {
                "highest": (confidence, img.copy()),
                "lowest": (confidence, img.copy()),
                "all_confidences": [(confidence, img.copy())]
            }
        else:
            if confidence > frame_collection[label]["highest"][0]:
                frame_collection[label]["highest"] = (confidence, img.copy())
            if confidence < frame_collection[label]["lowest"][0]:
                frame_collection[label]["lowest"] = (confidence, img.copy())
            frame_collection[label]["all_confidences"].append((confidence, img.copy()))
    
    # Add session ID to the frame
    cv2.putText(img, f"Session ID: {session_id}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    
    return img

def get_next_session_id(sheet):
    max_id = 0
    for row in sheet.iter_rows(min_row=2, max_col=1, values_only=True):
        if row[0] is not None:
            try:
                session_id = int(row[0])
                max_id = max(max_id, session_id)
            except ValueError:
                continue
    return max_id + 1

def update_log(numeric_session_id, session_id, start_time, end_time, label_confidence):
    file_name = "observation_logs.xlsx"
    
    # Load or create the workbook
    if os.path.exists(file_name):
        workbook = openpyxl.load_workbook(file_name)
    else:
        workbook = openpyxl.Workbook()
        workbook.active.append(["ID", "Session ID", "Date", "Start Time", "End Time", "Duration (seconds)", "Label", "Confidence Level", "Flag"])

    sheet = workbook.active
    
    # Calculate duration
    duration = (end_time - start_time).total_seconds()
    date = start_time.strftime("%Y-%m-%d")
    start_str = start_time.strftime("%H:%M:%S")
    end_str = end_time.strftime("%H:%M:%S")
    
    # Append session data for each label
    for label, (confidence, flag) in label_confidence.items():
        sheet.append([numeric_session_id, session_id, date, start_str, end_str, duration, label, f"{confidence}%", flag])
    
    # Save the workbook
    workbook.save(file_name)

def save_observation_frames(session_id, frame_collection):
    os.makedirs("observations", exist_ok=True)
    
    for label, frames in frame_collection.items():
        highest_confidence_frame = frames["highest"][1]
        lowest_confidence_frame = frames["lowest"][1]
        
        # Calculate average confidence frame
        all_confidences = frames["all_confidences"]
        average_confidence_frame = all_confidences[len(all_confidences) // 2][1]
        
        highest_filename = os.path.join("observations", f"{session_id}_{label}_H.jpg")
        lowest_filename = os.path.join("observations", f"{session_id}_{label}_L.jpg")
        average_filename = os.path.join("observations", f"{session_id}_{label}_A.jpg")
        
        cv2.imwrite(highest_filename, highest_confidence_frame)
        cv2.imwrite(lowest_filename, lowest_confidence_frame)
        cv2.imwrite(average_filename, average_confidence_frame)

# Create 'recordings' directory if it doesn't exist
os.makedirs("recordings", exist_ok=True)

# Create session ID based on the current timestamp
start_time = datetime.datetime.now()
session_id = start_time.strftime("%Y-%m-%d-%H-%M-%S")

# Create unique filename for recording
output_path = os.path.join("recordings", f"session_{session_id}.mp4")

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 20.0
frame_size = (640, 480)  # Match this to your webcam's resolution
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

# Dictionary to store label and confidence levels
label_confidence = {}
frame_collection = {}  # Dictionary to store frames based on confidence

# Load or create the workbook and get the next session ID
file_name = "observation_logs.xlsx"
if os.path.exists(file_name):
    workbook = openpyxl.load_workbook(file_name)
    sheet = workbook.active
    numeric_session_id = get_next_session_id(sheet)
else:
    numeric_session_id = 1

while True:
    ret, raw_frame = video_capture.read()
    if not ret:
        break

    # Resize frame to match the VideoWriter frame size
    raw_frame = cv2.resize(raw_frame, frame_size)

    # Create a copy of the frame for processing
    display_frame = raw_frame.copy()

    # Process frame (detection and drawing)
    display_frame = draw_boundary(display_frame, faceCascade, 1.3, 6, (255, 255, 255), clf, session_id, label_confidence, frame_collection)

    # Write raw frame to video file (without detection)
    out.write(raw_frame)

    # Display the resulting frame with detection
    cv2.imshow("face detection", display_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Capture end time
end_time = datetime.datetime.now()

# Update the observation log
update_log(numeric_session_id, session_id, start_time, end_time, label_confidence)

# Save observation frames
save_observation_frames(session_id, frame_collection)

# Release everything
video_capture.release()
out.release()
cv2.destroyAllWindows()

# 2. On recordings

In [8]:
import cv2
import os
from PIL import Image
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Function to draw boundary and recognize faces
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 76:
            if id == 1:
                cv2.putText(img, "Michelle", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
    
    return img

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Function to select and process a video file
def process_video():
    # Set up file dialog to select a video
    Tk().withdraw()  # Prevents an empty tkinter window from appearing
    file_path = askopenfilename(initialdir="recordings", title="Select a Recording",
                                filetypes=(("MP4 files", "*.mp4"), ("AVI files", "*.avi"), ("All files", "*.*")))
    
    if not file_path:
        print("No file selected. Exiting.")
        return
    
    video_capture = cv2.VideoCapture(file_path)
    
    while True:
        ret, img = video_capture.read()
        if not ret:
            break
        
        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

# Call the function to process video
process_video()

No file selected. Exiting.


In [10]:
import cv2
import os
from tkinter import Tk, filedialog

# Function to draw boundary and recognize faces
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 76:
            if id == 1:
                cv2.putText(img, "Michelle", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
    
    return img

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Function to select and process a video file
def process_video():
    # Set up DPI scaling
    root = Tk()
    root.withdraw()  # Prevents the main tkinter window from appearing

    # Increase DPI awareness
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)
    except Exception as e:
        print(f"Failed to set DPI awareness: {e}")

    initial_dir = os.path.join(os.getcwd(), "recordings")
    file_path = filedialog.askopenfilename(initialdir=initial_dir, 
                                           title="Select a Recording for Face Detection",
                                           filetypes=(("MP4 files", "*.mp4"), ("AVI files", "*.avi"), ("All files", "*.*")))
    
    if not file_path:
        print("No file selected. Exiting.")
        return
    
    video_capture = cv2.VideoCapture(file_path)
    
    while True:
        ret, img = video_capture.read()
        if not ret:
            break
        
        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

# Call the function to process video
process_video()


No file selected. Exiting.
